In [1]:
//importing spark packages for this task

import org.apache.spark.sql.SparkSession
import spark.implicits._


//create a datafram by reading in the large csv file

val df = spark.read.format("csv").option("header", "true").load("LCLoan.csv")


//show a summary of our target variable

df.groupBy("loan_status").count().show()

Intitializing Scala interpreter ...

Spark Web UI available at http://75cd00854c39:4040
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1558754856679)
SparkSession available as 'spark'


2019-05-25 03:27:35,033 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2019-05-25 03:27:57,605 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
+--------------------+-------+
|         loan_status|  count|
+--------------------+-------+
|          Fully Paid|1041952|
|             Default|     31|
|     In Grace Period|   8952|
|Does not meet the...|   1988|
|         Charged Off| 261654|
|            Oct-2015|      1|
|  Late (31-120 days)|  21897|
|             Current| 919695|
|Does not meet the...|    761|
|   Late (16-30 days)|   3737|
+--------------------+-------+



import org.apache.spark.sql.SparkSession
import spark.implicits._
df: org.apache.spark.sql.DataFrame = [id: string, member_id: string ... 143 more fields]


In [2]:
//created a subset of our large dataset
//at 1% of our data, this should result in about 20,000 rows

var subsetDF = df.sample(false, 0.01)



//display a summary of our target variable, 
//and ensure that none of the counts are too out-of-line with the large dataset

subsetDF.groupBy("loan_status").count().show()

+--------------------+-----+
|         loan_status|count|
+--------------------+-----+
|          Fully Paid|10449|
|             Default|    1|
|     In Grace Period|  100|
|Does not meet the...|   19|
|         Charged Off| 2666|
|  Late (31-120 days)|  208|
|             Current| 9184|
|Does not meet the...|    4|
|   Late (16-30 days)|   42|
+--------------------+-----+



subsetDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, member_id: string ... 143 more fields]


In [29]:
//Write a Folder that contains a new csv file. (it will be called part-0000-...)

subsetDF.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("SubsetLCLoan")

In [3]:

//lastl we'll ned to rename the file to start EDAs.

subsetDF.columns



res2: Array[String] = Array(id, member_id, loan_amnt, funded_amnt, funded_amnt_inv, term, int_rate, installment, grade, sub_grade, emp_title, emp_length, home_ownership, annual_inc, verification_status, issue_d, loan_status, pymnt_plan, url, desc, purpose, title, zip_code, addr_state, dti, delinq_2yrs, earliest_cr_line, inq_last_6mths, mths_since_last_delinq, mths_since_last_record, open_acc, pub_rec, revol_bal, revol_util, total_acc, initial_list_status, out_prncp, out_prncp_inv, total_pymnt, total_pymnt_inv, total_rec_prncp, total_rec_int, total_rec_late_fee, recoveries, collection_recovery_fee, last_pymnt_d, last_pymnt_amnt, next_pymnt_d, last_credit_pull_d, collections_12_mths_ex_med, mths_since_last_major_derog, policy_code, application_type, annual_inc_joint, dti_joint, verificati...

In [5]:
subsetDF.groupBy("id").count().show()
subsetDF.groupBy("member_id").count().show()


+----+-----+
|  id|count|
+----+-----+
|null|22673|
+----+-----+

+---------+-----+
|member_id|count|
+---------+-----+
|     null|22673|
+---------+-----+



In [6]:
subsetDF.na.drop()

res5: org.apache.spark.sql.DataFrame = [id: string, member_id: string ... 143 more fields]


In [8]:
subsetDF = subsetDF.drop("id")

subsetDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [member_id: string, loan_amnt: string ... 142 more fields]


In [13]:
subsetDF.filter(df("member_id").isNull || df("member_id") === "" || df("member_id").isNaN).count()

res11: Long = 22673


In [16]:
subsetDF.columns.toSeq

res14: Seq[String] = WrappedArray(loan_amnt, funded_amnt, funded_amnt_inv, term, int_rate, installment, grade, sub_grade, emp_title, emp_length, home_ownership, annual_inc, verification_status, issue_d, loan_status, pymnt_plan, url, desc, purpose, title, zip_code, addr_state, dti, delinq_2yrs, earliest_cr_line, inq_last_6mths, mths_since_last_delinq, mths_since_last_record, open_acc, pub_rec, revol_bal, revol_util, total_acc, initial_list_status, out_prncp, out_prncp_inv, total_pymnt, total_pymnt_inv, total_rec_prncp, total_rec_int, total_rec_late_fee, recoveries, collection_recovery_fee, last_pymnt_d, last_pymnt_amnt, next_pymnt_d, last_credit_pull_d, collections_12_mths_ex_med, mths_since_last_major_derog, policy_code, application_type, annual_inc_joint, dti_joint, verification_status...

In [17]:
//this ended up not deleting any columns after ~50 mintes  find a new method
//for(colname <- subsetDF.columns.toSeq){
//    if(subsetDF.filter(df(colName).isNull || df(colName) === "" || df(colName).isNaN).count() > 10000){
//        subsetDF = subsetDF.drop(colName)
//    }
//}

In [18]:
subsetDF

res16: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [loan_amnt: string, funded_amnt: string ... 141 more fields]
